In [1]:
# !unzip /home/aistudio/data/data78992/lcqmc.zip -d /home/aistudio/data/
# !unzip /home/aistudio/data/data78992/paws-x-zh.zip -d /home/aistudio/data/
# !unzip /home/aistudio/data/data78992/bq_corpus.zip -d /home/aistudio/data/

## 1. Load dataset

In [2]:
def load_dataset(fpath, num_row_skip=0):

    def read(fp):
        data = open(fp)

        for _ in range(num_row_skip):
            next(data)

        if "test" in fp:
            for line in data:
                line = line.strip().split('\t')
                yield line[0], line[1]
        else:
            for line in data:
                line = line.strip().split('\t')
                if len(line) == 3:
                    yield line[0], line[1], int(line[2])

    if isinstance(fpath, str):
        return list(read(fpath))
    elif isinstance(fpath, (list, tuple)):
        return [list(read(fp)) for fp in fpath]
    else:
        raise TypeError("Input fpath must be a str or a list/tuple of str")

In [3]:
train_set, dev_set, test_set = load_dataset(['./data/paws-x-zh/train.tsv', './data/paws-x-zh/dev.tsv', './data/paws-x-zh/test.tsv'])
# len(train_set), len(dev_set), len(test_set)
train_set = train_set + dev_set

## 2. Transform text

In [4]:
from paddlenlp.datasets import MapDataset
from paddle.io import BatchSampler, DataLoader
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.transformers import ErnieModel as SeqClfModel
from paddlenlp.transformers import ErnieTokenizer as PTMTokenizer
import numpy as np


MODEL_NAME = "ernie-1.0"
tokenizer = PTMTokenizer.from_pretrained(MODEL_NAME)


def example_converter(example, max_seq_length, tokenizer):
    text_a, text_b, label = example
    encoded = tokenizer(text=text_a, text_pair=text_b, max_seq_len=max_seq_length)
    input_ids = encoded["input_ids"]
    token_type_ids = encoded["token_type_ids"]
    label = np.array([label], dtype="int64")
    return input_ids, token_type_ids, label


def get_trans_fn(max_seq_length=128, tokenizer=tokenizer):
    return lambda ex: example_converter(ex, max_seq_length, tokenizer)


batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id), 
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    Stack(dtype="int64")
    ): fn(samples)


def create_dataloader(dataset, 
                      trans_fn, 
                      batchify_fn, 
                      test=False,
                      batch_size=128, 
                      shuffle=True, 
                      sampler=BatchSampler):
    
    if test:
        dataset = [d + (0,) for d in dataset]

    if not isinstance(dataset, MapDataset):
        dataset = MapDataset(dataset)
        
    dataset.map(trans_fn)
    batch_sampler = sampler(dataset, 
                            shuffle=shuffle, 
                            batch_size=batch_size)
    
    dataloder = DataLoader(dataset, 
                           batch_sampler=batch_sampler, 
                           collate_fn=batchify_fn)
    
    return dataloder

[2022-03-25 08:23:31,760] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/vocab.txt


In [5]:
max_seq_length = 128; batch_size = 64
trans_fn = get_trans_fn(max_seq_length)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn, batch_size=batch_size)
# dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn, batch_size=batch_size)
test_loader = create_dataloader(test_set, trans_fn, batchify_fn, shuffle=False, test=True, batch_size=batch_size)

## 3. Model building

In [6]:
from paddle import nn
import paddle


class PTM(nn.Layer):

    def __init__(self, pretrained_model, dropout=0.1, num_class=2):
        super().__init__()

        self.ptm = pretrained_model
        ptm_out_dim = self.ptm.config["hidden_size"]
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(ptm_out_dim, num_class)

    def encoder(self, input_ids, token_type_ids):
        _, embd = self.ptm(input_ids, token_type_ids)
        embd = self.dropout(embd)
        return embd

    def forward(self, input_ids, token_type_ids):
        embd = self.encoder(input_ids, token_type_ids)
        logits = self.fc1(embd)
        return logits

In [7]:
from paddlenlp.transformers import LinearDecayWithWarmup

epoch = 10
weight_decay = 0.0
warmup_proportion = 0.0
lr_scheduler = LinearDecayWithWarmup(5e-5, len(train_loader) * epoch,
                                         warmup_proportion)

def get_model(model):
    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
    optimizer = paddle.optimizer.AdamW(
    parameters=model.parameters(), 
    learning_rate=lr_scheduler, 
    weight_decay=weight_decay, 
    apply_decay_param_fun=lambda x: x in decay_params)

    criterion = paddle.nn.CrossEntropyLoss()

    model = paddle.Model(model)
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [8]:
ptm = SeqClfModel.from_pretrained(MODEL_NAME)
model = PTM(ptm)
model = get_model(model)

[2022-03-25 08:23:31,799] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
W0325 08:23:31.802661  7797 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0325 08:23:31.807628  7797 device_context.cc:465] device: 0, cuDNN Version: 7.6.
[2022-03-25 08:23:37,869] [    INFO] - Weights from pretrained model not used in ErnieModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_norm.bias']


## 4. Model training

In [9]:
model.fit(train_loader, epochs=epoch, verbose=2, log_freq=100)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 100/799 - loss: 0.6375 - acc: 0.5906 - 396ms/step
step 200/799 - loss: 0.5679 - acc: 0.6795 - 397ms/step
step 300/799 - loss: 0.2788 - acc: 0.7279 - 396ms/step
step 400/799 - loss: 0.4233 - acc: 0.7579 - 396ms/step
step 500/799 - loss: 0.3408 - acc: 0.7760 - 396ms/step
step 600/799 - loss: 0.2779 - acc: 0.7916 - 396ms/step
step 700/799 - loss: 0.2776 - acc: 0.8031 - 397ms/step
step 799/799 - loss: 0.2029 - acc: 0.8116 - 397ms/step
Epoch 2/10
step 100/799 - loss: 0.2308 - acc: 0.9114 - 396ms/step
step 200/799 - loss: 0.1756 - acc: 0.9073 - 394ms/step
step 300/799 - loss: 0.0685 - acc: 0.9093 - 395ms/step
step 400/799 - loss: 0.2880 - acc: 0.9089 - 396ms/step
step 500/799 - loss: 0.2351 - acc: 0.9088 - 398ms/step
step 600/799 - loss: 0.3234 - acc: 0.9097 - 398ms/step
step 700/799 - loss: 0.2408 - acc: 0.9098 - 398ms/step
step 799/799 - loss: 0.1434 - acc: 0.9101 -

## 5. Prediction

In [10]:
import paddle.nn.functional as F


predictions = []
logits = model.predict(test_loader)

for batch in logits[0]:
    batch = paddle.to_tensor(batch)
    probs = F.softmax(batch, axis=1)
    preds = paddle.argmax(probs, axis=1).numpy().tolist()
    predictions.extend(preds)

Predict begin...
step 32/32 [==============================] - ETA: 6s - 229ms/st - ETA: 5s - 197ms/st - ETA: 4s - 187ms/st - ETA: 4s - 182ms/st - ETA: 3s - 179ms/st - ETA: 3s - 176ms/st - ETA: 3s - 175ms/st - ETA: 2s - 173ms/st - ETA: 2s - 173ms/st - ETA: 2s - 171ms/st - ETA: 1s - 171ms/st - ETA: 1s - 171ms/st - ETA: 1s - 171ms/st - ETA: 0s - 170ms/st - ETA: 0s - 167ms/st - 163ms/step          
Predict samples: 2000


In [11]:
with open('paws-x.tsv', 'w') as f:
    f.write("index\tprediction")
    for idx, p in enumerate(predictions):
        f.write(f"\n{idx}\t{p}")
    f.close()